In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
# pip install -q -U tensorflow-addons

In [3]:
# import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D,LocallyConnected1D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization,Add,concatenate
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [4]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [5]:
N_FEATURES = 10

In [6]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'Class_label']


In [7]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [8]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [9]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [10]:
 pd.options.display.float_format = "{:,.5f}".format

In [11]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [14]:
# import os
path='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/Total_process_TT/'

In [15]:
#  for i in range(num_res_net_blocks_64):
#     x = res_net_block_64(x, 64, 3)

In [16]:
# file_path_train=path+'Train_data'+'.'+'csv'
file_path_train=path+'nina_pro_naveen_DB1_Train'+'.'+'csv'
print(file_path_train)

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/Total_process_TT/nina_pro_naveen_DB1_Train.csv


In [17]:
file_path_test=path+'nina_pro_naveen_DB1_Test'+'.'+'csv'
# file_path_test=path+'Test_data'+'.'+'csv'
print(file_path_test)

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/Total_process_TT/nina_pro_naveen_DB1_Test.csv


In [18]:
df_Train=read_data_Train(file_path_train)
show_basic_dataframe_info(df_Train)
df_Train.head(5)

Number of columns in the dataframe: 11
Number of rows in the dataframe: 4914000



,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,Class_label
0,0.00720,0.00242,0.00493,0.00242,0.00245,0.00241,0.00308,0.01634,0.00241,0.00247,1.00000
1,0.00836,0.00243,0.00478,0.00242,0.00245,0.00241,0.00312,0.01699,0.00241,0.00247,1.00000
2,0.00949,0.00243,0.00464,0.00242,0.00246,0.00241,0.00317,0.01762,0.00241,0.00248,1.00000
3,0.01052,0.00243,0.00451,0.00242,0.00246,0.00241,0.00322,0.01822,0.00242,0.00248,1.00000
4,0.01142,0.00243,0.00439,0.00242,0.00246,0.00241,0.00327,0.01877,0.00242,0.00248,1.00000


In [19]:
df_Test=read_data_Test(file_path_test)
show_basic_dataframe_info(df_Test)
df_Test.head(5)

Number of columns in the dataframe: 11
Number of rows in the dataframe: 2106000



,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,Class_label
0,0.03176,0.00247,0.00243,0.00240,0.00244,0.00240,0.00257,0.03562,0.00244,0.00241,1.00000
1,0.03205,0.00248,0.00244,0.00240,0.00245,0.00240,0.00258,0.03567,0.00244,0.00241,1.00000
2,0.03234,0.00248,0.00245,0.00240,0.00246,0.00240,0.00259,0.03569,0.00244,0.00241,1.00000
3,0.03261,0.00249,0.00246,0.00240,0.00246,0.00240,0.00260,0.03570,0.00244,0.00241,1.00000
4,0.03285,0.00249,0.00246,0.00240,0.00246,0.00240,0.00261,0.03566,0.00245,0.00241,1.00000


In [20]:
pd.options.display.float_format = "{:,.5f}".format
df_Train.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,Class_label
0,0.00720,0.00242,0.00493,0.00242,0.00245,0.00241,0.00308,0.01634,0.00241,0.00247,1.00000
1,0.00836,0.00243,0.00478,0.00242,0.00245,0.00241,0.00312,0.01699,0.00241,0.00247,1.00000
2,0.00949,0.00243,0.00464,0.00242,0.00246,0.00241,0.00317,0.01762,0.00241,0.00248,1.00000
3,0.01052,0.00243,0.00451,0.00242,0.00246,0.00241,0.00322,0.01822,0.00242,0.00248,1.00000
4,0.01142,0.00243,0.00439,0.00242,0.00246,0.00241,0.00327,0.01877,0.00242,0.00248,1.00000


In [21]:
scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])

In [22]:
df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])

In [23]:
pd.options.display.float_format = "{:,.5f}".format
df_Train.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,Class_label
0,-0.67597,-0.64137,-0.51809,-0.43726,-0.37267,-0.45614,-0.76033,-0.90886,-0.51943,-0.77849,1.00000
1,-0.67360,-0.64137,-0.51846,-0.43725,-0.37263,-0.45613,-0.76027,-0.90759,-0.51942,-0.77848,1.00000
2,-0.67130,-0.64136,-0.51882,-0.43725,-0.37260,-0.45613,-0.76019,-0.90634,-0.51942,-0.77847,1.00000
3,-0.66919,-0.64136,-0.51915,-0.43724,-0.37256,-0.45613,-0.76012,-0.90516,-0.51942,-0.77846,1.00000
4,-0.66736,-0.64135,-0.51946,-0.43724,-0.37252,-0.45612,-0.76004,-0.90409,-0.51942,-0.77845,1.00000


In [24]:
pd.options.display.float_format = "{:,.5f}".format
df_Test.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,Class_label
0,0.03176,0.00247,0.00243,0.00240,0.00244,0.00240,0.00257,0.03562,0.00244,0.00241,1.00000
1,0.03205,0.00248,0.00244,0.00240,0.00245,0.00240,0.00258,0.03567,0.00244,0.00241,1.00000
2,0.03234,0.00248,0.00245,0.00240,0.00246,0.00240,0.00259,0.03569,0.00244,0.00241,1.00000
3,0.03261,0.00249,0.00246,0.00240,0.00246,0.00240,0.00260,0.03570,0.00244,0.00241,1.00000
4,0.03285,0.00249,0.00246,0.00240,0.00246,0.00240,0.00261,0.03566,0.00245,0.00241,1.00000


In [25]:
df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])

In [26]:
pd.options.display.float_format = "{:,.5f}".format
df_Test.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,Class_label
0,-0.62585,-0.64122,-0.52438,-0.43732,-0.37277,-0.45618,-0.76115,-0.87110,-0.51936,-0.77862,1.00000
1,-0.62526,-0.64121,-0.52435,-0.43732,-0.37266,-0.45618,-0.76114,-0.87102,-0.51936,-0.77862,1.00000
2,-0.62467,-0.64120,-0.52433,-0.43731,-0.37256,-0.45618,-0.76112,-0.87096,-0.51935,-0.77862,1.00000
3,-0.62412,-0.64119,-0.52432,-0.43731,-0.37252,-0.45618,-0.76110,-0.87095,-0.51935,-0.77862,1.00000
4,-0.62363,-0.64117,-0.52432,-0.43731,-0.37250,-0.45618,-0.76109,-0.87102,-0.51934,-0.77862,1.00000


In [27]:
LABEL = 'ActivityEncoded'
# Transform the labels from String to Integer via LabelEncoder
le = preprocessing.LabelEncoder()
# Add a new column to the existing DataFrame with the encoded values
df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
# df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())

In [28]:
# The number of steps within one time segment
TIME_PERIODS = 20
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 20

In [29]:
x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
print('x_train shape: ', x_train.shape)
# print(x_train)
print(x_train.shape[0], 'training samples')
print('y_train shape: ', y_train.shape)
# Set input & output dimensions
num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
print('num_time_periods',num_time_periods)
print('num_sensors',num_sensors)
num_classes = le.classes_.size
print('class_list',list(le.classes_))
# input_shape = (num_time_periods,num_sensors)
# print(input_shape)
input_shape = (num_time_periods,num_sensors)
#x_train = x_train.reshape(x_train.shape[0], input_shape)
print('x_train shape:', x_train[0].shape)
print('input_shape:', input_shape)
x_train = x_train.astype('float32')
# x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
# y_train = y_train.astype('float32')
# print(y_train)
y_train_hot = np_utils.to_categorical(y_train, num_classes)
print(y_train_hot)
# y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
print('New y_train shape: ', y_train_hot.shape)

x_train shape:  (245699, 20, 10)
245699 training samples
y_train shape:  (245699,)
num_time_periods 20
num_sensors 10
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0]
x_train shape: (20, 10)
input_shape: (20, 10)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
New y_train shape:  (245699, 52)


In [30]:
x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
print('x_test shape: ', x_test.shape)
# print(x_train)
print(x_test.shape[0], 'testing samples')
print('y_test shape: ', y_test.shape)
# Set input_shape / reshape for Keras
#x_test = x_test.reshape(x_test.shape[0], input_shape)
x_test = x_test.astype('float32')
y_test = y_test.astype('float32')
y_test_hot = np_utils.to_categorical(y_test, num_classes)

x_test shape:  (105299, 20, 10)
105299 testing samples
y_test shape:  (105299,)


In [31]:
# n_steps, n_length = 20, 25
# n_steps, n_length= 10, 50
# n_steps, n_length= 16, 32
n_length =  20
n_depth=10
n_channel=10
x_train = x_train.reshape(x_train.shape[0], n_length,n_depth)
print('x_train shape: ', x_train.shape)
# x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
# print('x_valid shape: ', x_valid.shape)
x_test = x_test.reshape(x_test.shape[0],  n_length,n_depth)
print('x_test shape: ', x_test.shape)
n_outputs = y_train_hot.shape[1]
print('n_outputs',n_outputs)

x_train shape:  (245699, 20, 10)
x_test shape:  (105299, 20, 10)
n_outputs 52


In [66]:
def conv_block(input_data,n_channel):
    inputs=input_data
    print(inputs.shape)
    l_y=[]
    for i in range(0,n_channel):
        ip=inputs[:,:,i]
        print('ip_shape',ip.shape)
#         ip1=tf.convert_to_tensor(ip, dtype=tf.float32)
        ip=tf.reshape(ip,(tf.shape(ip)[0],20,1))
        print('ip_shape',ip.shape)
        x = Conv1D(64, kernel_size=3,kernel_initializer="he_normal",kernel_regularizer=l1(1e-04), \
               padding='same',input_shape=ip.shape)(ip)
        print('conv1_o',x.shape)
        x = BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x = Activation('relu')(x)
        x= Conv1D(filters=64, kernel_size=3,padding="same", kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1)(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        y= Dropout(0.5)(x)
        l_y.append(y)
    y_stack=tf.stack(l_y, axis=2)   
    return y_stack

In [67]:
def generate_model():
    inputs = Input(shape=(n_length,n_depth),name="main")
    p=conv_block(inputs,10)
    print(p.shape)
    print(type(p))
#     print(y.shape)
    y=Flatten()(p)
    print(y.shape)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(128, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    outputs=Dense(n_outputs, activation='softmax')(y)
    print(outputs.shape)
#     wei_mscnn_model = Model(inputs,sub_inputs], outputs)
    print('inputs_shape',inputs.shape)
    final_model = Model(inputs,outputs,name="wei_mscnn_model")
    return final_model        

In [68]:
if __name__ == "__main__":
    model = generate_model()

(None, 20, 10)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
ip_shape (None, 20)
ip_shape (None, 20, 1)
conv1_o (None, 20, 64)
(None, 20, 10, 64)
<class 'tensorflow.python.framework.ops.Tensor'>
(None, 12800)
(None, 52)
inputs_shape (None, 20, 10)


In [69]:
verbose, epochs, batch_size = 0, 200, 1000

In [70]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 50.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [71]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [72]:
adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
# sgd=tfa.optimizers.SGDW(weight_decay=0.0001,learning_rate=0.1, momentum=0.9, nesterov=False, name='SGDW')
sgd=tf.optimizers.SGD(learning_rate=1e-3, momentum=0.9, nesterov=False, name='SGD')
checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_20X10/checkpoint.hdf5'
# model.load_weights(checkpoint_filepath) 
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)

Model: "wei_mscnn_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main (InputLayer)               [(None, 20, 10)]     0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_40 (T [(None, 20)]         0           main[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_strided_slice_42 (T [(None, 20)]         0           main[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_strided_slice_44 (T [(None, 20)]         0           main[0][0]                       
____________________________________________________________________________________

In [73]:
csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_20X10/CNN_nina_20X10.csv', append=True, separator=';')
history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)

Epoch 1/200
246/246 [==============================] - ETA: 0s - loss: 4.3958 - accuracy: 0.1058
Epoch 00001: val_accuracy improved from -inf to 0.18215, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_20X10/checkpoint.hdf5
246/246 [==============================] - 22s 90ms/step - loss: 4.3958 - accuracy: 0.1058 - val_loss: 3.5115 - val_accuracy: 0.1821 - lr: 0.0010
Epoch 2/200
246/246 [==============================] - ETA: 0s - loss: 3.4481 - accuracy: 0.1682
Epoch 00002: val_accuracy improved from 0.18215 to 0.22726, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_20X10/checkpoint.hdf5
246/246 [==============================] - 19s 79ms/step - loss: 3.4481 - accuracy: 0.1682 - val_loss: 3.1287 - val_accuracy: 0.2273 - lr: 0.0010
Epoch 3/200
246/246 [==============================] - ETA: 0s - loss: 3.2168 - accuracy: 0.1992
Epoch 00003: val_accuracy improved from 0.22726 to 0.26753, saving model to /media/naveen/nav/ma

Epoch 44/200
246/246 [==============================] - ETA: 0s - loss: 2.1047 - accuracy: 0.4408
Epoch 00044: val_accuracy did not improve from 0.54337
246/246 [==============================] - 18s 73ms/step - loss: 2.1047 - accuracy: 0.4408 - val_loss: 1.8039 - val_accuracy: 0.5385 - lr: 0.0010
Epoch 45/200
246/246 [==============================] - ETA: 0s - loss: 2.0973 - accuracy: 0.4413
Epoch 00045: val_accuracy did not improve from 0.54337
246/246 [==============================] - 18s 74ms/step - loss: 2.0973 - accuracy: 0.4413 - val_loss: 1.8082 - val_accuracy: 0.5416 - lr: 0.0010
Epoch 46/200
246/246 [==============================] - ETA: 0s - loss: 2.0966 - accuracy: 0.4418
Epoch 00046: val_accuracy did not improve from 0.54337
246/246 [==============================] - 18s 74ms/step - loss: 2.0966 - accuracy: 0.4418 - val_loss: 1.8049 - val_accuracy: 0.5417 - lr: 0.0010
Epoch 47/200
246/246 [==============================] - ETA: 0s - loss: 2.0919 - accuracy: 0.4433
Epoch

Epoch 91/200
246/246 [==============================] - ETA: 0s - loss: 1.8475 - accuracy: 0.4963
Epoch 00091: val_accuracy did not improve from 0.57945
246/246 [==============================] - 18s 74ms/step - loss: 1.8475 - accuracy: 0.4963 - val_loss: 1.6111 - val_accuracy: 0.5791 - lr: 1.0000e-04
Epoch 92/200
246/246 [==============================] - ETA: 0s - loss: 1.8455 - accuracy: 0.4968
Epoch 00092: val_accuracy did not improve from 0.57945
246/246 [==============================] - 18s 75ms/step - loss: 1.8455 - accuracy: 0.4968 - val_loss: 1.6144 - val_accuracy: 0.5763 - lr: 1.0000e-04
Epoch 93/200
246/246 [==============================] - ETA: 0s - loss: 1.8460 - accuracy: 0.4955
Epoch 00093: val_accuracy did not improve from 0.57945
246/246 [==============================] - 18s 75ms/step - loss: 1.8460 - accuracy: 0.4955 - val_loss: 1.6093 - val_accuracy: 0.5782 - lr: 1.0000e-04
Epoch 94/200
246/246 [==============================] - ETA: 0s - loss: 1.8449 - accuracy: 

Epoch 116/200
246/246 [==============================] - ETA: 0s - loss: 1.8186 - accuracy: 0.5024
Epoch 00116: val_accuracy did not improve from 0.58530
246/246 [==============================] - 19s 79ms/step - loss: 1.8186 - accuracy: 0.5024 - val_loss: 1.5850 - val_accuracy: 0.5844 - lr: 1.0000e-05
Epoch 117/200
246/246 [==============================] - ETA: 0s - loss: 1.8171 - accuracy: 0.5024
Epoch 00117: val_accuracy did not improve from 0.58530
246/246 [==============================] - 19s 77ms/step - loss: 1.8171 - accuracy: 0.5024 - val_loss: 1.5960 - val_accuracy: 0.5814 - lr: 1.0000e-05
Epoch 118/200
246/246 [==============================] - ETA: 0s - loss: 1.8173 - accuracy: 0.5026
Epoch 00118: val_accuracy did not improve from 0.58530
246/246 [==============================] - 20s 79ms/step - loss: 1.8173 - accuracy: 0.5026 - val_loss: 1.5869 - val_accuracy: 0.5835 - lr: 1.0000e-05
Epoch 119/200
246/246 [==============================] - ETA: 0s - loss: 1.8162 - accura

KeyboardInterrupt: 

In [ ]:
best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
print('epoch_number',best_index+1)
print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])

In [ ]:
model.load_weights(checkpoint_filepath) 
_, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
print('test_accuracy',testaccuracy)
test_acc.append(testaccuracy)
print(test_acc)

In [ ]:
statistics.mean(test_acc)